In [1]:
import sys
sys.path.append("/planning_through_contact")

In [2]:
import numpy as np
import pandas as pd
import plotly
import os
from examples.planar_hand.contact_sampler import PlanarHandContactSampler

from qsim.parser import QuasistaticParser

from irs_rrt.irs_rrt import IrsNode
from irs_rrt.irs_rrt_projection import IrsRrtProjection
from irs_rrt.rrt_params import IrsRrtProjectionParams, SmoothingMode
from examples.planar_hand.planar_hand_setup import *

from irs_mpc2.quasistatic_visualizer import QuasistaticVisualizer


In [3]:

# %% quasistatic system
q_parser = QuasistaticParser(q_model_path)

q_vis = QuasistaticVisualizer.make_visualizer(q_parser)
q_sim, q_sim_py = q_vis.q_sim, q_vis.q_sim_py
plant = q_sim.get_plant()

dim_x = q_sim.num_dofs()
dim_u = q_sim.num_actuated_dofs()
idx_a_l = plant.GetModelInstanceByName(robot_l_name)
idx_a_r = plant.GetModelInstanceByName(robot_r_name)
idx_u = plant.GetModelInstanceByName(object_name)
contact_sampler = PlanarHandContactSampler(
    q_sim=q_sim, q_sim_py=q_sim_py, pinch_prob=0.1
)

q_u0 = np.array([0.0, 0.2, 0])
q0_dict = contact_sampler.calc_enveloping_grasp(q_u0)
q0 = q_sim.get_q_vec_from_dict(q0_dict)

joint_limits = {
    idx_u: np.array([[-0.3, 0.3], [0.3, 0.5], [-0.01, np.pi + 0.01]])
}

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [4]:

# %% RRT testing
rrt_params = IrsRrtProjectionParams(q_model_path, joint_limits)
rrt_params.h = h
rrt_params.smoothing_mode = SmoothingMode.k1AnalyticPyramid
rrt_params.log_barrier_weight_for_bundling = 100
rrt_params.root_node = IrsNode(q0)
rrt_params.max_size = 600
rrt_params.goal = np.copy(q0)
rrt_params.goal[2] = np.pi
rrt_params.termination_tolerance = 0.01
rrt_params.goal_as_subgoal_prob = 0.3
rrt_params.regularization = 1e-4
rrt_params.rewire = False
rrt_params.grasp_prob = 0.
rrt_params.distance_threshold = np.inf
rrt_params.stepsize = 0.35
rrt_params.distance_metric = "local_u"

# params.distance_metric = 'global'  # If using global metric
rrt_params.global_metric = np.array([10.0, 10.0, 1.0, 0.1, 0.1, 0.1, 0.1])

print("start state:", q0)
print("goal state:", rrt_params.goal)

start state: [ 0.          0.2         0.         -1.04694189 -1.32075746  1.04694189
  1.32075746]
goal state: [ 0.          0.2         3.14159265 -1.04694189 -1.32075746  1.04694189
  1.32075746]


In [5]:
rrt_params.use_free_solvers = True
# contact_sampler.sim_params.use_free_solvers = True

In [6]:
prob_rrt = IrsRrtProjection(rrt_params, contact_sampler, q_sim, q_sim_py)
prob_rrt.iterate()

100%|█████████▉| 599/600 [00:43<00:00, 13.79it/s]


In [12]:
(
    q_knots_trimmed,
    u_knots_trimmed,
) = prob_rrt.get_trimmed_q_and_u_knots_to_goal()
q_vis.publish_trajectory(q_knots_trimmed, h=rrt_params.h)
print("trajectory size", len(q_knots_trimmed))

closest distance to subgoal 547.0428134483942
trajectory size 5


In [13]:
d_batch = prob_rrt.calc_distance_batch(rrt_params.goal)
print("minimum distance: ", d_batch.min())

# %%
node_id_closest = np.argmin(d_batch)

# %%
prob_rrt.save_tree(f"planar_hand_tree_{rrt_params.max_size}_{0}.pkl")


# %%
closest_state = prob_rrt.q_matrix[node_id_closest, :]
print("closest state", closest_state)
print("goal state", rrt_params.goal)


minimum distance:  547.0428134483942
closest state [ 1.37604323e-04  2.20727302e-01  9.38476484e-02 -9.99674936e-01
 -1.18394045e+00  6.32655276e-01  9.98752544e-01]
goal state [ 0.          0.2         3.14159265 -1.04694189 -1.32075746  1.04694189
  1.32075746]


In [32]:
tasks = ["90_rotation", "180_rotation", "box_throw"]
start_states = {
    "90_rotation": np.array([0,0.2,0,0,0,0,0]),
    "180_rotation": np.array([0,0.2,0,0,0,0,0]),
    "box_throw": np.array([0.05,0.2,0,0,0,0,0]),
}
goal_states = {
    "90_rotation": np.array([0.1, 0.3, np.pi/2, 0,0,0,0]),
    "180_rotation": np.array([0.1, 0.3, np.pi, 0,0,0,0]),
    "box_throw": np.array([0.82, 0.47, 0, 0,0,0,0]),
}
metrics = {
    "90_rotation": np.array([1, 1, 1 / 2 * np.pi, 0, 0, 0, 0]),
    "180_rotation": np.array([1, 1, 1 / 2 * np.pi, 0, 0, 0, 0]),
    "box_throw": np.array([5, 10, 0, 0, 0, 0, 0]),
}

In [36]:
def compute_progress(prob_rrt, start_state, goal_state, metric):
    rrt_params = prob_rrt.rrt_params
    max_size = rrt_params.max_size
    progress = np.zeros(max_size)
    prob_rrt.iterate()

    delta = start_state - goal_state
    distance_normalizer = delta.T @ np.diag(metric) @ delta


    for i in range(max_size):
        delta = prob_rrt.q_matrix[i] - goal_state
        distance = delta.T @ np.diag(metric) @ delta
        progress[i] = 1 - distance / distance_normalizer
        print(delta)
    # for i in range(1, max_size):
    #     if progress[i-1] > progress[i]:
    #         progress[i] = progress[i-1]
    return progress

def set_start_goal_states(contact_sampler, rrt_params, qu_start, qu_goal):
    q0_dict = contact_sampler.calc_enveloping_grasp(qu_start)
    q0 = q_sim.get_q_vec_from_dict(q0_dict)
    rrt_params.root_node = IrsNode(q0)
    rrt_params.goal = np.copy(q0)
    rrt_params.goal[0:2] = qu_goal
    return rrt_params


    

In [ ]:
# rrt_iterations = 7000
# num_samples = 5

task = "90_rotation"
rrt_iterations = 20
num_samples = 2
progress = np.zeros(num_samples, rrt_iterations)

rrt_params.max_size = rrt_iterations
rrt_params = set_start_goal_states(contact_sampler, rrt_params, start_states[task][0:2], goal_states[task][0:2])

prob_rrt = IrsRrtProjection(rrt_params, contact_sampler, q_sim, q_sim_py)
progress = compute_progress(prob_rrt, start_states[task], goal_states[task], metrics[task])
print("progress", progress)



In [ ]:

# Convert matrices to DataFrame
df_progress = pd.DataFrame(progress)

# Save DataFrames to CSV files

# Ensure the directory exists
benchmark_dir = "benchmark_data"
os.makedirs(benchmark_dir, exist_ok=True)

df_progress.to_csv(f"benchmark_data/progress_{task_name}.csv", index=False)
df_closest_node_angular_errors.to_csv("benchmark_data/closest_node_angular_errors.csv", index=False)

In [34]:

import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(y=progress, mode='lines'))
fig.update_layout(title='RRT planner ', xaxis_title='RRT Iteration', yaxis_title='Progress')
fig.show()